In [1]:

import yaml

import jax
import brax
from brax.io import html

from rl_games.torch_runner import Runner
from rl_games.envs.brax import BraxEnv

from IPython.display import HTML, IFrame, display, clear_output 

In [2]:
config_name = 'rl_games/configs/brax/ppo_humanoid.yaml'
trained_network = 'nn/Humanoid_brax.pth'

In [3]:
with open(config_name, 'r') as stream:
    config = yaml.safe_load(stream)
    runner = Runner()
    runner.load(config)
    config = runner.get_prebuilt_config()
    agent = runner.create_player()
    agent.restore(trained_network)
env_config = config['env_config']

{'observation_space': Box(-inf, inf, (299,), float32), 'action_space': Box(-1.0, 1.0, (17,), float32), 'agents': 1, 'value_size': 1}
build mlp: 299
sigma
actor_mlp.0.weight
actor_mlp.0.bias
actor_mlp.2.weight
actor_mlp.2.bias
actor_mlp.4.weight
actor_mlp.4.bias
value.weight
value.bias
mu.weight
mu.bias
RunningMeanStd:  (299,)
=> loading checkpoint 'nn/Humanoid_brax.pth'


/home/trrrrr/anaconda3/envs/rlgpu/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
num_actors = 1
env = BraxEnv('', num_actors, **env_config)

/home/trrrrr/anaconda3/envs/rlgpu/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [5]:

qps = []
obs = env.reset()
total_reward = 0
num_steps = 0
state = env.state
while not state.done:
    qps.append(state.qp)
    act = agent.get_action(obs)
    obs, reward, is_done, info = env.step(act.unsqueeze(0))
    total_reward += reward.item()
    num_steps += 1
    if is_done.item():
        break
    state = env.state

print('Total Reward: ', total_reward)
print('Num steps: ', num_steps) 


Total Reward:  7067.2854170799255
Num steps:  704


In [6]:
def visualize(sys, qps):
    return HTML(html.render(sys, qps))

In [10]:
visualize(env.env.sys, qps)
